In [296]:
# %%
import pandas as pd
import weasyprint
import webbrowser
import datetime
import os
import yfinance as yf
import math
import matplotlib.pyplot as plt


def highlight_values(x):

    if isinstance(x, (int, float)):
        if x < 0:
            return 'color: red'
        elif x >= 0:
            return 'color: black'
        else:
            pass



def scan_for_csv_files(directory):
    """Scans a directory for CSV files and returns their paths.

    Args:
        directory: The directory to scan.

    Returns:
        A list of paths to CSV files found in the directory.
    """

    csv_files = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            csv_file_path = os.path.join(directory, file)
            csv_files.append(csv_file_path)
    return csv_files

files = scan_for_csv_files(os.getcwd())
count = 0

def getData(ticker, key):
    try:
        x = str(ticker)
        sym = ""
        if (x[-1:]) == ".":
            sym += x + "L"
        else:
            sym += x + ".L"
        stock = yf.Ticker(sym)
        stock= stock.info

        return stock[key]

    except:
        pass


def getSector(ticker):
    x = getData(ticker, 'sector')
    return x
def getDividendYield(ticker):
    x =getData(ticker, 'dividendYield')
    if not x ==  None:
        return ("{:.2f}".format(x*100))
    else:
        pass




# for csv in files:



df = pd.read_csv("shares.csv")
pd.set_option('display.width', 300)




df.columns = df.columns.str.strip('\ufeff')
df['Qty'] = df['Qty'].astype('Int64').astype('string')

df = df.drop(columns=['Day Gain/Loss', 'Day Gain/Loss %', 'Market Value £'])
df["Gain/Loss %"] = df["Gain/Loss %"].str.rstrip("%").astype('float')

totals = df.tail(2)
df.drop(df.tail(2).index,inplace=True)







In [36]:
df['Sector'] = df['Symbol'].apply(getSector)

In [37]:
df["Dividend Yield"] = df['Symbol'].apply(getDividendYield)

In [297]:
df = df.sort_values(by="Gain/Loss %", ascending=False)

In [298]:
df

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
39,RR.,Rolls-Royce Holdings,564,475.90p,"£2,684.08",£507.18,"£2,176.90",429.22,89.93p
46,W7L,Warpaint London,143,550.00p,£786.50,£447.08,£339.42,75.92,312.64p
23,IDS,International Distribution Services,502,339.00p,"£1,701.78",£998.09,£703.69,70.50,198.82p
41,TW.,Taylor Wimpey,727,161.40p,"£1,173.38",£798.90,£374.48,46.87,109.89p
42,TSCO,Tesco,1352,362.80p,"£4,905.06","£3,359.93","£1,545.13",45.99,248.52p
14,SMDS,DS Smith,1055,472.60p,"£4,985.93","£3,439.94","£1,545.99",44.94,326.06p
22,IAG,International Consolidated Airlines Group SA,519,182.90p,£949.25,£655.52,£293.73,44.81,126.30p
36,PFD,Premier Foods,1554,179.00p,"£2,781.66","£1,967.59",£814.07,41.37,126.61p
1,ABF,Associated British Foods,95,"2,490.00p","£2,365.50","£1,703.76",£661.74,38.84,"1,793.43p"
4,BA.,BAE Systems,1001,"1,316.00p","£13,173.16","£9,537.75","£3,635.41",38.12,952.82p


In [299]:

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 39 to 37
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Symbol         46 non-null     object 
 1   Name           49 non-null     object 
 2   Qty            49 non-null     string 
 3   Price          49 non-null     object 
 4   Market Value   49 non-null     object 
 5   Book Cost      49 non-null     object 
 6   Gain/Loss      49 non-null     object 
 7   Gain/Loss %    49 non-null     float64
 8   Average Price  49 non-null     object 
dtypes: float64(1), object(7), string(1)
memory usage: 3.8+ KB


In [281]:
# df['Market Value'] = df['Market Value'].str.replace('£', '')
# df['Market Value'] = df['Market Value'].str.replace(',', '')
# df['Market Value'] = df['Market Value'].astype('float')


In [300]:
result = pd.concat([df,totals])
# result = result.fillna('')



In [303]:
result['Market Value'] = result['Market Value'].str.replace('£', '')
result['Market Value'] = result['Market Value'].str.replace(',', '')
result['Market Value'] = result['Market Value'].astype('float')

In [305]:
result

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
39,RR.,Rolls-Royce Holdings,564,475.90p,2684.08,£507.18,"£2,176.90",429.22,89.93p
46,W7L,Warpaint London,143,550.00p,786.50,£447.08,£339.42,75.92,312.64p
23,IDS,International Distribution Services,502,339.00p,1701.78,£998.09,£703.69,70.50,198.82p
41,TW.,Taylor Wimpey,727,161.40p,1173.38,£798.90,£374.48,46.87,109.89p
42,TSCO,Tesco,1352,362.80p,4905.06,"£3,359.93","£1,545.13",45.99,248.52p
14,SMDS,DS Smith,1055,472.60p,4985.93,"£3,439.94","£1,545.99",44.94,326.06p
22,IAG,International Consolidated Airlines Group SA,519,182.90p,949.25,£655.52,£293.73,44.81,126.30p
36,PFD,Premier Foods,1554,179.00p,2781.66,"£1,967.59",£814.07,41.37,126.61p
1,ABF,Associated British Foods,95,"2,490.00p",2365.50,"£1,703.76",£661.74,38.84,"1,793.43p"
4,BA.,BAE Systems,1001,"1,316.00p",13173.16,"£9,537.75","£3,635.41",38.12,952.82p


In [306]:
result['Gain/Loss']= result["Gain/Loss"].str.replace('£', '')
result['Gain/Loss']= pd.to_numeric(result["Gain/Loss"].str.replace(',', '') ,errors='coerce')



AttributeError: Can only use .str accessor with string values!

In [307]:

result['Gain/Loss %']= pd.to_numeric(result["Gain/Loss %"],errors='coerce')


In [308]:
result

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
39,RR.,Rolls-Royce Holdings,564,475.90p,2684.08,£507.18,2176.90,429.22,89.93p
46,W7L,Warpaint London,143,550.00p,786.50,£447.08,339.42,75.92,312.64p
23,IDS,International Distribution Services,502,339.00p,1701.78,£998.09,703.69,70.50,198.82p
41,TW.,Taylor Wimpey,727,161.40p,1173.38,£798.90,374.48,46.87,109.89p
42,TSCO,Tesco,1352,362.80p,4905.06,"£3,359.93",1545.13,45.99,248.52p
14,SMDS,DS Smith,1055,472.60p,4985.93,"£3,439.94",1545.99,44.94,326.06p
22,IAG,International Consolidated Airlines Group SA,519,182.90p,949.25,£655.52,293.73,44.81,126.30p
36,PFD,Premier Foods,1554,179.00p,2781.66,"£1,967.59",814.07,41.37,126.61p
1,ABF,Associated British Foods,95,"2,490.00p",2365.50,"£1,703.76",661.74,38.84,"1,793.43p"
4,BA.,BAE Systems,1001,"1,316.00p",13173.16,"£9,537.75",3635.41,38.12,952.82p


In [309]:
result = result.reset_index(drop=True)
result = result.fillna('')
# Drop rows with NaN values


In [310]:
result

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
0,RR.,Rolls-Royce Holdings,564,475.90p,2684.08,£507.18,2176.9,429.22,89.93p
1,W7L,Warpaint London,143,550.00p,786.5,£447.08,339.42,75.92,312.64p
2,IDS,International Distribution Services,502,339.00p,1701.78,£998.09,703.69,70.5,198.82p
3,TW.,Taylor Wimpey,727,161.40p,1173.38,£798.90,374.48,46.87,109.89p
4,TSCO,Tesco,1352,362.80p,4905.06,"£3,359.93",1545.13,45.99,248.52p
5,SMDS,DS Smith,1055,472.60p,4985.93,"£3,439.94",1545.99,44.94,326.06p
6,IAG,International Consolidated Airlines Group SA,519,182.90p,949.25,£655.52,293.73,44.81,126.30p
7,PFD,Premier Foods,1554,179.00p,2781.66,"£1,967.59",814.07,41.37,126.61p
8,ABF,Associated British Foods,95,"2,490.00p",2365.5,"£1,703.76",661.74,38.84,"1,793.43p"
9,BA.,BAE Systems,1001,"1,316.00p",13173.16,"£9,537.75",3635.41,38.12,952.82p


In [311]:
result['Gain/Loss'] = pd.to_numeric(result['Gain/Loss'], errors='coerce')
result['Gain/Loss %'] = pd.to_numeric(result['Gain/Loss %'], errors='coerce')

# result['Market Value'] = result['Market Value'].str.replace('£', '')


result['Market Value'] = pd.to_numeric(result['Market Value'], errors='coerce')


In [312]:
result

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
0,RR.,Rolls-Royce Holdings,564,475.90p,2684.08,£507.18,2176.90,429.22,89.93p
1,W7L,Warpaint London,143,550.00p,786.50,£447.08,339.42,75.92,312.64p
2,IDS,International Distribution Services,502,339.00p,1701.78,£998.09,703.69,70.50,198.82p
3,TW.,Taylor Wimpey,727,161.40p,1173.38,£798.90,374.48,46.87,109.89p
4,TSCO,Tesco,1352,362.80p,4905.06,"£3,359.93",1545.13,45.99,248.52p
5,SMDS,DS Smith,1055,472.60p,4985.93,"£3,439.94",1545.99,44.94,326.06p
6,IAG,International Consolidated Airlines Group SA,519,182.90p,949.25,£655.52,293.73,44.81,126.30p
7,PFD,Premier Foods,1554,179.00p,2781.66,"£1,967.59",814.07,41.37,126.61p
8,ABF,Associated British Foods,95,"2,490.00p",2365.50,"£1,703.76",661.74,38.84,"1,793.43p"
9,BA.,BAE Systems,1001,"1,316.00p",13173.16,"£9,537.75",3635.41,38.12,952.82p


In [313]:

# Apply styling with proper formatting
def custom_format(x):
    if (isinstance(x, str)):
        return x
    elif pd.isna(x):  # Check if the value is NaN
        return '' # Return an empty string for NaN
    elif isinstance(x, (int, float)):  # Check if it's a number
        return f'£{x:.2f}' if 'Gain/Loss' in result.columns else f'{x:.2f}%' if 'Gain/Loss %' in result.columns else f'£{x:.2f}'
    return x  # Default return for other types

# Apply styling with proper formatting using a conditional function
styled = result.style.format({
    'Gain/Loss': custom_format,
    'Gain/Loss %': lambda x: f'{x:.2f}%' if pd.notna(x) else '',
    'Market Value': custom_format,
}).applymap(highlight_values, subset=['Gain/Loss'])

/tmp/ipykernel_12389/498042319.py:16: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  }).applymap(highlight_values, subset=['Gain/Loss'])


In [315]:
styled

,Symbol,Name,Qty,Price,Market Value,Book Cost,Gain/Loss,Gain/Loss %,Average Price
0,RR.,Rolls-Royce Holdings,564,475.90p,£2684.08,£507.18,£2176.90,429.22%,89.93p
1,W7L,Warpaint London,143,550.00p,£786.50,£447.08,£339.42,75.92%,312.64p
2,IDS,International Distribution Services,502,339.00p,£1701.78,£998.09,£703.69,70.50%,198.82p
3,TW.,Taylor Wimpey,727,161.40p,£1173.38,£798.90,£374.48,46.87%,109.89p
4,TSCO,Tesco,1352,362.80p,£4905.06,"£3,359.93",£1545.13,45.99%,248.52p
5,SMDS,DS Smith,1055,472.60p,£4985.93,"£3,439.94",£1545.99,44.94%,326.06p
6,IAG,International Consolidated Airlines Group SA,519,182.90p,£949.25,£655.52,£293.73,44.81%,126.30p
7,PFD,Premier Foods,1554,179.00p,£2781.66,"£1,967.59",£814.07,41.37%,126.61p
8,ABF,Associated British Foods,95,"2,490.00p",£2365.50,"£1,703.76",£661.74,38.84%,"1,793.43p"
9,BA.,BAE Systems,1001,"1,316.00p",£13173.16,"£9,537.75",£3635.41,38.12%,952.82p


In [16]:
sector_market_value = df.groupby('Sector')['Market Value'].sum()

In [17]:
sector_market_value

Sector
Basic Materials            1009.76
Communication Services      722.57
Consumer Cyclical         16360.70
Consumer Defensive        11645.57
Energy                     3689.80
Financial Services         8569.29
Healthcare                 1749.11
Industrials               25117.65
Technology                   20.53
Utilities                  2922.69
Name: Market Value, dtype: float64

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(sector_market_value, labels=sector_market_value.index, autopct='%1.1f%%', startangle=90)
plt.title('Market Value Proportion by Sector')
# ')  # Equal aspect ratio ensures the pie chart is circular
plt.show()

In [317]:

HTML_TEMPLATE1 = '''
<html>
<head>
<style>
@page {
    size: landscape;
    margin: 0in 0in 0in 0in;

}
body{
    display:flex;
    flex-direction: row-reverse;
    margin:0;
}
div{
    position: relative;

    font-size: 7px;
    width: 10%;
    font-family:  	Courier New;
    padding-top: 10px;

    font-weight: bold;

}
table {
    margin-left: 0;
    margin-right: 0;
    margin-top:0;
    font-size:10px;
}
table, th, td {
    border: 0.2px solid black;
    border-collapse: collapse;
}
th, td {
    padding: 2px;
    text-align: left;
    font-family: Helvetica, Arial, sans-serif;

}
table tbody tr:hover {
    background-color: #dddddd;
}
table thead th {
    text-align: center;
}
.wide {
    width: 90%;
}
</style>
</head>
<body>
'''

HTML_TEMPLATE2 = '''
</body>
</html>
'''


def to_html_pretty(df, filename='out.html', title=''):

    ht = ''
    if title != '':
        ht += '<div> %s </div>\n' % title
    ht += df.to_html(classes='wide', escape=False, index=False)

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(HTML_TEMPLATE1 + ht + HTML_TEMPLATE2)

# Pretty print the dataframe as an html table to a file
intermediate_html = 'intermediate.html'

x = datetime.datetime.now()
title = 'Updated: ' + x.strftime('%b') + ' ' + x.strftime('%y')
to_html_pretty(styled,intermediate_html, title )
# if you do not want pretty printing, just use pandas:
# result.to_html(intermediate_html)

# Convert the html file to a pdf file using weasyprint

out_pdf= os.path.basename(os.path.normpath('shares.csv')).replace('.csv', '') + '.pdf'
weasyprint.HTML(intermediate_html).write_pdf(out_pdf)


#  webbrowser.open_new_tab("/tmp/intermediate.html")

weasyprint.HTML('intermediate.html').write_pdf('out_styled.pdf')
# This is the table pretty printer used above: